In [1]:
import os
print(os.getcwd())
os.chdir('../../../')
print(os.getcwd())

/mnt/c/Users/luiso/OneDrive/Documents/double_descent_project_C++/experiments/gridsearch/EURUSD
/mnt/c/Users/luiso/OneDrive/Documents/double_descent_project_C++


In [ ]:
import argparse
import subprocess
from typing import Tuple, Dict, Any

import optuna


def run_cpp(
    bin_path: str,
    lags: int,
    window: int,
    sigma: float,
    kfolds: int,
    extra_args: list[str],
) -> Tuple[float, float]:
    cmd = [
        bin_path,
        "--lags", str(lags),
        "--window", str(window),
        "--sigma", str(sigma),
        "--kfolds", str(kfolds),
        *extra_args,
    ]

    try:
        out = subprocess.check_output(cmd, text=True, stderr=subprocess.PIPE).strip()
    except subprocess.CalledProcessError as e:
        # If your C++ returns non-zero, treat as a failed trial.
        # You can inspect e.stderr if needed.
        raise RuntimeError(f"C++ run failed. stderr:\n{e.stderr}") from e

    parts = out.split()
    if len(parts) < 1:
        raise RuntimeError(f"Unexpected stdout from C++ (empty). Got: {out!r}")

    mean_mse = float(parts[0])
    mean_var = float(parts[1]) if len(parts) > 1 else float("nan")
    return mean_mse, mean_var


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--bin", required=True, help="Path to C++ binary, e.g. ./abo_solver")
    parser.add_argument("--n_trials", type=int, default=200)
    parser.add_argument("--kfolds", type=int, default=3)
    parser.add_argument("--storage", default="", help="e.g. sqlite:///optuna.db (optional)")
    parser.add_argument("--study_name", default="abo_hpo")
    parser.add_argument("--seed", type=int, default=123)
    parser.add_argument(
        "--var_max",
        type=float,
        default=float("inf"),
        help="Constraint: penalize if mean_var exceeds this (optional).",
    )
    parser.add_argument(
        "--penalty",
        type=float,
        default=1000.0,
        help="Penalty multiplier if mean_var > var_max.",
    )

    # Search spaces (edit to match your experiment)
    parser.add_argument("--lags", default="15,30,60", help="Comma-separated categorical values")
    parser.add_argument("--windows", default="90,120,240", help="Comma-separated categorical values")
    parser.add_argument("--sigma_min", type=float, default=0.5)
    parser.add_argument("--sigma_max", type=float, default=10.0)
    parser.add_argument("--sigma_log", action="store_true", help="Log-uniform for sigma")

    # Any extra args you want forwarded to C++ (data path, mode, etc.)
    # Example:
    #   python tune_abo_optuna.py ... --extra --data data/eurusd.csv --mode val
    parser.add_argument("--extra", nargs=argparse.REMAINDER, default=[])

    args = parser.parse_args()

    lags_space = [int(x.strip()) for x in args.lags.split(",") if x.strip()]
    win_space = [int(x.strip()) for x in args.windows.split(",") if x.strip()]
    extra_args = args.extra

    sampler = optuna.samplers.TPESampler(seed=args.seed)

    if args.storage:
        study = optuna.create_study(
            direction="minimize",
            study_name=args.study_name,
            storage=args.storage,
            load_if_exists=True,
            sampler=sampler,
        )
    else:
        study = optuna.create_study(direction="minimize", sampler=sampler)

    def objective(trial: optuna.Trial) -> float:
        lags = trial.suggest_categorical("lags", lags_space)
        window = trial.suggest_categorical("window", win_space)

        if args.sigma_log:
            sigma = trial.suggest_float("sigma", args.sigma_min, args.sigma_max, log=True)
        else:
            sigma = trial.suggest_float("sigma", args.sigma_min, args.sigma_max)

        mean_mse, mean_var = run_cpp(
            bin_path=args.bin,
            lags=lags,
            window=window,
            sigma=sigma,
            kfolds=args.kfolds,
            extra_args=extra_args,
        )

        # Save extra metrics for inspection (these show up in Optuna UI / logs)
        trial.set_user_attr("mean_var", mean_var)

        # Optional: constraint by penalty
        if mean_var == mean_var and mean_var > args.var_max:  # mean_var==mean_var checks not-NaN
            mean_mse = mean_mse + args.penalty * (mean_var - args.var_max)

        return mean_mse

    study.optimize(objective, n_trials=args.n_trials)

    print("\n=== BEST ===")
    print("Best value (objective):", study.best_value)
    print("Best params:", study.best_params)

    # Print the best mean_var (if recorded)
    best_trial = study.best_trial
    if "mean_var" in best_trial.user_attrs:
        print("Best trial mean_var:", best_trial.user_attrs["mean_var"])

    print("\nTip: re-run your C++ binary once with best params and --mode test on held-out test folds.")


if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: ipykernel_launcher.py [-h] --bin BIN [--n_trials N_TRIALS]
                             [--kfolds KFOLDS] [--storage STORAGE]
                             [--study_name STUDY_NAME] [--seed SEED]
                             [--var_max VAR_MAX] [--penalty PENALTY]
                             [--lags LAGS] [--windows WINDOWS]
                             [--sigma_min SIGMA_MIN] [--sigma_max SIGMA_MAX]
                             [--sigma_log] [--extra ...]
ipykernel_launcher.py: error: the following arguments are required: --bin


SystemExit: 2

/root/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
